# Confronto tra l’algoritmo classico di RProp e le sue varianti per la classificazione di immagini MNIST

Questo quaderno Jupyter offre un ambiente interattivo per l’addestramento e l’analisi di reti neurali applicate alla classificazione delle cifre del dataset MNIST, permettendo agli utenti di esplorare diverse configurazioni di reti e algoritmi di ottimizzazione, con particolare focus sulle varianti dell’algoritmo Rprop.

## Sommario
    
    1. Estrazione e Preparazione del Dataset MNIST
	2. Configurazione delle Reti Neurali
	3. Addestramento con Varianti di Rprop
	4. Valutazione delle Prestazioni con Grafici
	5. Test Finale sul Set di Test

## Configurazioni delle Reti Neurali

Gli utenti possono personalizzare la struttura delle reti neurali scegliendo tra diverse configurazioni, inclusa la possibilità di definire:

	• Numero di strati nascosti.
	• Numero di neuroni per strato.
	• Funzioni di attivazione per ciascun livello.
	
## Personalizzazione dei Parametri di Addestramento

Il quaderno permette una flessibile impostazione dei parametri di addestramento, quali:

	• Numero di epoche.
	• Tasso di apprendimento.
	• Suddivisione del set di addestramento.
	•Numero di ripetizioni, per eseguire più volte l’addestramento con una configurazione fissa, consentendo una valutazione più accurata degli algoritmi.

## Algoritmi di Ottimizzazione Rprop

Sono implementate diverse varianti dell’algoritmo di ottimizzazione Rprop, che possono essere confrontate in termini di efficienza e prestazioni durante l’addestramento delle reti neurali.

## Valutazione delle Prestazioni delle Reti Neurali

Il quaderno fornisce un’analisi completa delle prestazioni delle reti, con le seguenti metriche:

	• Errore sui set di addestramento e validazione.
	• Accuratezza sui set di addestramento, validazione e test.
	• Tempo di esecuzione complessivo.
	• Media e varianza delle prestazioni su più ripetizioni, per valutare la stabilità e la consistenza degli algoritmi di ottimizzazione e delle configurazioni di rete.

In [7]:
"""
    1. Estrazione e Preparazione del Dataset MNIST
"""

from nndlpy import MnistDataset as MnistDataset

import numpy as np
import pandas as pd

"""
    Init
"""
# Carica il dataset MNIST CSV con pandas
train_data = pd.read_csv('data/mnist_train.csv')
test_data = pd.read_csv('data/mnist_test.csv')

# Converti i dati in array con numpy
train_array = np.array(train_data)
test_array = np.array(test_data)

# Definisci le dimensioni del training set
train_set_size = 10000  # Almeno 10.000 campioni per il training
test_set_size = 2500    # Almeno 2.500 campioni per il test

"""
    Validation Set
"""
# Mescola i dati di training casualmente
np.random.shuffle(train_array)
# Calcola l'indice per il validation set (esempio: il 20% del training set)
val_index = int(np.ceil(train_set_size * 0.20))
# Estrai il validation set dal training array
validation_array = train_array[:val_index - 1]
# Estrai il validation set
validation_X, validation_Y = MnistDataset.get_mnist_validation(validation_array)

"""
    Training Set
"""
# Estrai il training set rimanente (senza i dati di validation)
train_array_reduced = train_array[val_index:train_set_size]
# Estrai il training set (dopo aver rimosso i dati di validazione)
train_X, train_Y = MnistDataset.get_mnist_training(train_array_reduced)

"""
    Test Set
"""
# Estrai il test set direttamente dal file di test
test_X, test_Y = MnistDataset.get_mnist_test(test_array, test_set_size)

In [8]:
"""
    2. Configurazione delle Reti Neurali
"""

from nndlpy import ActivationFunctions as ActivationFunctions
from nndlpy import LossFunctions as LossFunctions
from nndlpy import NeuralNetwork as NeuralNetwork

# Parametri di addestramento
epochs = 35
learning_rate = 0.00001
num_of_networks = 5  # Definisci quante reti testare (maggiore di 0)

# Funzioni di attivazione per i layer nascosti
hidden_activation_funcs = [ActivationFunctions.relu]
# Funzione di attivazione per l'output
output_activation_func = ActivationFunctions.identity
#Funzione di perdita per l'output
loss_func = LossFunctions.cross_entropy_softmax

# Numero di neuroni per ciascun layer nascosto
hidden_layers_sizes = [32]

# Dimensione dell'input e dell'output (numero di neuroni nel primo e ultimo layer)
input_layer_size = train_X.shape[0]
output_layer_size = train_Y.shape[0]

# Creazione e inizializzazione delle reti neurali
neural_networks = []

for i in range(num_of_networks):
    try:
        # Inizializza una nuova rete neurale con la configurazione corrente
        network = NeuralNetwork.NeuralNetwork(
            hidden_activation_funcs,
            output_activation_func,
            loss_func,
            input_layer_size,
            hidden_layers_sizes,
            output_layer_size
        )
        print(f"Rete neurale {i+1} inizializzata con successo.")
    except ValueError as e:
        print(f"Errore durante l'inizializzazione della rete {i+1}: {e}")
        continue
    
    # Aggiungi la rete neurale alla lista di reti
    neural_networks.append(network)

    # Modifica l'architettura rimuovendo l'ultima funzione di attivazione per la prossima rete (se necessario)
    # if hidden_activation_funcs:
    #     hidden_activation_funcs = hidden_activation_funcs[:-1]
    
# Mostra la struttura della prima rete
print("\nArchitettura della prima rete:")
neural_networks[0].get_network() 

Rete neurale 1 inizializzata con successo.
Rete neurale 2 inizializzata con successo.
Rete neurale 3 inizializzata con successo.
Rete neurale 4 inizializzata con successo.
Rete neurale 5 inizializzata con successo.

Architettura della prima rete:
Numero di layer nascosti: 1
Dimensione dell'input: 784
Dimensione dell'output: 10
Neuroni nei layer nascosti: [32]
Funzioni di attivazione: relu, identity
Funzione di perdita: cross_entropy_softmax


In [9]:
# Inizializza una lista vuota per memorizzare i risultati
std_metrics_list = []
std_trained_nets = []

for i in range(num_of_networks):
    std_training_net = neural_networks[i].clone_network()

    print('\n\n\nRun numero', i + 1, '\n')
    # Batch training Rprop standard
    std_metrics = std_training_net.train(train_X, train_Y, validation_X, validation_Y, epochs=epochs, learning_rate=learning_rate, rprop_type="STANDARD")
    std_trained_nets.append(std_training_net)
    std_metrics_list.append(std_metrics)




Run numero 1 

Epoca: 1/35  Rprop utilizzata: STANDARD
    Accuratezza Training: 0.11725,  Perdita Training: 18466.35097;
    Accuratezza Validation: 0.10105,  Perdita Validation: 4627.83601

Epoca: 2/35  Rprop utilizzata: STANDARD
    Accuratezza Training: 0.1315,  Perdita Training: 18272.38767;
    Accuratezza Validation: 0.11806,  Perdita Validation: 4580.58974

Epoca: 3/35  Rprop utilizzata: STANDARD
    Accuratezza Training: 0.24038,  Perdita Training: 41426.83668;
    Accuratezza Validation: 0.23712,  Perdita Validation: 10621.45609

Epoca: 4/35  Rprop utilizzata: STANDARD
    Accuratezza Training: 0.27925,  Perdita Training: 63471.67855;
    Accuratezza Validation: 0.29865,  Perdita Validation: 15149.41774

Epoca: 5/35  Rprop utilizzata: STANDARD
    Accuratezza Training: 0.18212,  Perdita Training: 66701.01069;
    Accuratezza Validation: 0.1906,  Perdita Validation: 16623.30696

Epoca: 6/35  Rprop utilizzata: STANDARD
    Accuratezza Training: 0.38925,  Perdita Training: 59